In [2]:
import pandas as pd                 # dataframe storing
from bs4 import BeautifulSoup       # parsing web content in a nice way
import ssl                          # MAY BE UNNECESSARY: provides access to the security socket layer (ssl) https://docs.python.org/3/library/ssl.html
import urllib                       # open and navigate URL's
import os

from selenium import webdriver      # load and run the webpage dynamically.
#from selenium.webdriver.chrome.options import Options



In [42]:
# Use a autonomous Chrome page to dynamically load the page for scraping. 
# Requires webdriver to be downloaded and then its path directed to.
# Unless you want to change to location, make sure the chromedriver program is located within the same file folder that you run this application in.
# You must have chrome (or a chromium base like internet explorer). Download software here: https://chromedriver.chromium.org/downloads
path = os.getcwd() + "/chromedriver"
driver = webdriver.Chrome(executable_path = path)

homeURL = "https://ehrafworldcultures.yale.edu/"
searchTokens = "/search?q=text%3AApple&fq=culture_level_samples%7CPSF"

# Load the HTML page (note that this should be updated to allow for modular input)
driver.get(homeURL + searchTokens)

# Find then click on each tab to reveal content for scraping
# Elements must be individually clicked backwards. I do not know why this is a thing but my guess is each 
# clicked tab adds HTML pushing future tabs to a new location thereby making some indexing no longer point to a retrieved tab. 
# Loading backwards avoids this.
country_tab = driver.find_elements_by_class_name('trad-overview__result')
for i in range(len(country_tab)-1,-1,-1):
    try:
        country_tab[i].click()
    except:
        print(f"WARNING tab {i} failed to be clicked")

# Parse processed webpage with BeautifulSoup
soup = BeautifulSoup(driver.page_source)



In [22]:
#Example of finding then printing out the dynamic webpage (before clicking). Shown here are the continents
continent_dir = soup.find_all('div',
{'class':'trad-overview__result'})

for x in continent_dir:
    print(x.h4.button.text)


Africa
Asia
Europe
Middle America and the Caribbean
Middle East
North America
Oceania
South America


In [43]:
# Create a dictionary to store all cultures and their links for later use
culture_dict = {}

# find the tables containing the cultures then loop through them to extract their subregion, continent, name, and the link to the passages
# Note that if the ehraf website changes, this loop might need fixing by changing where the information is retrieved.
# Also note that if the dynamic page is not loaded correctly, (a warning is given above), this may also fail.
table_culture_links = soup.find_all('tr', {'class':'mdc-data-table__row'})
for culture_i in table_culture_links:
    culture_list = list(culture_i.children)

    subRegion = culture_list[0].text
    cultureName = culture_list[1].a.text
    link = culture_list[1].a.attrs['href']
    continent = culture_i.findParent('table', {'role':'region'}).attrs['id']
    # 
    culture_dict[cultureName] = {"Continent":continent, "SubRegion":subRegion, "link":link}
print(f"Number of cultures extracted {len(culture_dict)}")

Number of cultures extracted 48


In [41]:

for key, val in culture_dict.items():
    #navigate to a culture's page via the link
    driver.get(homeURL + culture_dict[key]['link'])

    #click through the source tabs
    sourceTab = driver.find_elements_by_class_name('mdc-data-table__row')
    for i in range(len(sourceTab)-1,-1,-1):
        try:
            sourceTab[i].click()
        except:
            print(f"WARNING sourceTab {i} failed to be clicked")
    
    #Log the source table's results number in order to know where to start and stop clicking.
    # Skip every 2 logs as they do not contain the information desired
    sourceCount = soup.find_all('td',{'class':'mdc-data-table__cell mdc-data-table__cell--numeric'})
    sourceCount_list = list(map(lambda x: int(x.text), sourceCount[0::3]))

    #go through each source
    # pastTabs is to help with indexing so the clicker does not click tabs already clicked
    pastTabs = 0
    for i in sourceCount_list:
        total = i #superfluous
        while total > 0:
            #mark out the number of tabs that will be clicked
            if total >= 10:
                log = 10
            else:
                log = total
            total = total - log

            
            docTab = driver.find_elements_by_class_name('trad-data__results--row')
            for i in range(log+pastTabs-1,pastTabs-1,-1):
                try:
                    docTab[i].click()
                except:
                    print(f"WARNING docTab {i} failed to be clicked")
            




        
# driver.get(

WARNING sourceTab 0 failed to be clicked


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=53588): Max retries exceeded with url: /session/9559a0ea5136f8c1d137f5f2842295b2/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fed5995fa90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [62]:
log = 6
pastTabs = 0
x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
for i in range(log+pastTabs-1,pastTabs-1,-1):
    print(i)

5
4
3
2
1
0


In [31]:
source_table = soup.find_all('td',{'class':'mdc-data-table__cell--numeric'})
source_table

[<td class="mdc-data-table__cell mdc-data-table__cell--numeric">1</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1926</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">not specified</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">4</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1937</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1926-1929</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1956</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">not specified</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">2</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">1971</td>,
 <td class="mdc-data-table__cell mdc-data-table__cell--numeric">ca. 1863 - 1930</td>]

In [61]:
#A way to retrieve the results list
source_table = soup.find_all('td',{'class':'mdc-data-table__cell mdc-data-table__cell--numeric'})
source_table_list = list(map(lambda x: int(x.text), source_table[0::3]))
print(source_table_list)
# for i in source_table[0::3]:
#     print(i.text)


[1, 4, 1, 2]


In [44]:
#Dummy example for clicking links ion the next page.
driver.get(homeURL + culture_dict['Azande']['link'])
sourceTab = driver.find_elements_by_class_name('mdc-data-table__row')
for i in range(len(sourceTab)-1,-1,-1):
    try:
        sourceTab[i].click()
    except:
        print(f"WARNING tab {i} failed to be clicked")
        
soup = BeautifulSoup(driver.page_source)

In [27]:
driver.get(homeURL + culture_dict['Azande']['link'])

In [25]:
resultsTab = driver.find_elements_by_class_name('trad-data__results--row')
resultsTab

[<selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="e52c8c44-d05f-4048-b7bd-19e4645e4d80")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="3908a8fb-f2cb-4e38-ade6-637a072040a3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="6980db78-7e48-4e65-88cb-926f9ca0cace")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="90b49251-6d02-4dbb-a4e9-ab625d722592")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="86ff4a41-da13-4ec5-acc4-b52b44d4a6a9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="13281286-645b-4ca5-9889-899d1ecf1f3f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="6051bcac-f673-4fa8-bd1f-f8

In [26]:
resultsTab[0]

<selenium.webdriver.remote.webelement.WebElement (session="1eeb63906685dccc9e96c7be4121e53d", element="e52c8c44-d05f-4048-b7bd-19e4645e4d80")>

In [17]:
x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
print(x[1:11])
print(len(x[1:11]))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
10


In [8]:
document_table = soup.find_all('tr',{'class':'mdc-data-table__row'})
len(document_table)

4

In [9]:
# import bs4.element as element
# body_child_tags = list(filter( lambda x: type(x) == element.Tag, country_links)) #go through, if there are tags, create a list then print
# body_child_tags 